In [1]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import timm
import torch

# sns.set_theme()

%matplotlib widget
%load_ext autoreload
%autoreload 2

In [2]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

torch.manual_seed(42)

Need a model that takes a sequence of images as input and chooses two among them for the best exposure fusion result

In [3]:
from math import comb

import pandas as pd
import torchvision as tv
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import transforms

from dhdrnet.dataset import CachingDataset
from dhdrnet.free_choice_model import FreeChoiceDHDRNet, fit

exposure_vals = [-5, -3, -1, 0, 1, 3, 5]
ds = CachingDataset(
    data_dir=Path("../../data"),
    exposure_values=exposure_vals,
    transform=transforms.Compose(
        [transforms.ToPILImage(), transforms.Resize((300, 300)), transforms.ToTensor()]
    ),
)
# dl = DataLoader(ds, batch_size=3, shuffle=True, num_workers=12, pin_memory=True)

In [4]:
# for batch in dl:
#     images, ev_classes, scores = batch
#     print(f"{images.shape=}")
#     print(f"{ev_classes}")
#     break

datasets are being processed by the dataloader correctly :)

In [5]:
train_ds = Subset(ds, range(20))
val_ds = Subset(ds, range(20, 40))

In [6]:
num_classes = comb(len(exposure_vals), 2)
model = FreeChoiceDHDRNet(num_classes)
fit(
    model,
    train_ds,
    val_ds,
    batch_size=10,
    num_epochs=10,
    exp_name=4,
    logdir="../../logs",
)

Train:   0%|          | 0/2 [00:28<?, ?it/s, train_loss=3.11, batch_idx=0]

Val:   0%|          | 0/1 [00:00<?, ?it/s]

Val:   0%|          | 0/1 [04:52<?, ?it/s, val_loss=3.07, batch_idx=0]

Val: 100%|██████████| 1/1 [04:52<00:00, 292.33s/it, val_loss=3.07, batch_idx=0]

                                                                               
Train:   0%|          | 0/2 [00:02<?, ?it/s, train_loss=2.53, batch_idx=0]

Val:   0%|          | 0/1 [00:00<?, ?it/s]

Val:   0%|          | 0/1 [00:03<?, ?it/s, val_loss=3.01, batch_idx=0]

Val: 100%|██████████| 1/1 [00:03<00:00,  3.76s/it, val_loss=3.01, batch_idx=0]

                                                                              
Train:   0%|          | 0/2 [00:02<?, ?it/s, train_loss=1.97, batch_idx=0]

Val:   0%|          | 0/1 [00:00<?, ?it/s]

Val:   0%|          | 0/1 [00:03<?, ?it/s, val_loss=2.87, batch_idx=0]

Val: 100%|██████████| 1/1 [00:03<00:00,  3.79s/it, val_loss=2.87, batch_idx=0]

                        